In [10]:
%matplotlib notebook
%run 'utils.ipynb'

In [26]:
df = create_clean_df('cifar')
df = df.sort_values(['agg_method', 'agg_period', 'agg_prob', 'num_workers']).reset_index(drop=True)

In [27]:
df['exp_id']

0    cifar10_elastic_gossip_aggprob512_alpha05_4
1    cifar10_elastic_gossip_aggprob128_alpha05_4
2     cifar10_elastic_gossip_aggprob32_alpha05_4
3      cifar10_elastic_gossip_aggprob8_alpha05_4
4              cifar10_gopr_aggprob512_alpha05_4
5              cifar10_gopr_aggprob128_alpha05_4
6               cifar10_gopr_aggprob32_alpha05_4
7                cifar10_gopr_aggprob8_alpha05_4
8                                  cifar10_gar_4
9                                   cifar_single
Name: exp_id, dtype: object

In [28]:
df

,agg.AggTestAcc,agg.Rank,agg.hostname,agg.level,agg.msg,agg.name,agg.pid,agg.time,agg.v,__version__,...,rank_2.v,rank_3.Rank,rank_3.TestAcc,rank_3.hostname,rank_3.level,rank_3.msg,rank_3.name,rank_3.pid,rank_3.time,rank_3.v
0,0.8983,0.0,preemptible-2gpu-instance1,30.0,,Rank-0,2401.0,2018-06-27T00:01:22Z,0.0,0.2.0,...,0.0,3.0,0.8910,preemptible-2gpu-instance1,30.0,,Rank-3,2400.0,2018-06-27T00:01:03Z,0.0
1,0.9044,0.0,dist-training-multi-gpu-1-preemptible,30.0,,Rank-0,4723.0,2018-06-25T03:42:13Z,0.0,0.2.0,...,0.0,3.0,0.9025,dist-training-multi-gpu-1-preemptible,30.0,,Rank-3,4725.0,2018-06-25T03:42:02Z,0.0
2,0.9139,0.0,preemptible-2gpu-instance2,30.0,,Rank-0,2397.0,2018-06-27T15:09:13Z,0.0,0.2.0,...,0.0,3.0,0.9149,preemptible-2gpu-instance2,30.0,,Rank-3,2399.0,2018-06-27T15:08:53Z,0.0
3,0.9146,0.0,preemptible-2gpu-instance1,30.0,,Rank-0,2800.0,2018-06-27T18:58:33Z,0.0,0.2.0,...,0.0,3.0,0.9156,preemptible-2gpu-instance1,30.0,,Rank-3,2799.0,2018-06-27T18:58:13Z,0.0
4,0.8845,0.0,preemptible-2gpu-instance2,30.0,,Rank-0,12015.0,2018-06-27T00:08:26Z,0.0,0.2.0,...,0.0,3.0,0.8858,preemptible-2gpu-instance2,30.0,,Rank-3,12017.0,2018-06-27T00:08:06Z,0.0
5,0.9050,0.0,preemptible-2gpu-instance1,30.0,,Rank-0,2940.0,2018-06-27T01:50:58Z,0.0,0.2.0,...,0.0,3.0,0.8984,preemptible-2gpu-instance1,30.0,,Rank-3,2938.0,2018-06-27T01:50:39Z,0.0
6,0.9065,0.0,preemptible-2gpu-instance1,30.0,,Rank-0,2399.0,2018-06-27T14:43:50Z,0.0,0.2.0,...,0.0,3.0,0.9032,preemptible-2gpu-instance1,30.0,,Rank-3,2402.0,2018-06-27T14:43:30Z,0.0
7,0.9135,0.0,preemptible-2gpu-instance2,30.0,,Rank-0,2449.0,2018-06-27T18:58:53Z,0.0,0.2.0,...,0.0,3.0,0.9132,preemptible-2gpu-instance2,30.0,,Rank-3,2451.0,2018-06-27T18:58:33Z,0.0
8,0.9193,0.0,preemptible-2gpu-instance2,30.0,,Rank-0,25537.0,2018-06-27T02:31:11Z,0.0,0.2.0,...,0.0,3.0,0.9196,preemptible-2gpu-instance2,30.0,,Rank-3,25540.0,2018-06-27T02:30:53Z,0.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
def get_initials(s):
    s = s.strip().split()
    s = map(lambda x: x[0], s)
    return ''.join(s)


case_1 = df \
    .query('num_workers >= 4') \
    [['agg_method', 'num_workers', 'agg_prob', 'elastic_alpha', 'rank_0.TestAcc', 'exp_id', 'agg.AggTestAcc']] \
    .query('(agg_method != "gossipingSgd") or (agg_prob < 0.5)') \
    .query('(elastic_alpha == 0.5) or (agg_method != "elasticGossip")') \
    .sort_values(['num_workers', 'agg_prob', 'elastic_alpha'], ascending=[True, False, True]) \
    .assign(elastic_alpha=lambda x: 
            x.apply(lambda r: r.elastic_alpha if r.agg_method == 'elasticGossip' else None, axis=1)) \
    .assign(agg_prob=lambda x:
            x.apply(lambda r: r.agg_prob if (r.agg_method in ['elasticGossip', 'gossipingSgd']) else None, axis=1)) \
    .assign(agg_method=lambda x:
            x.agg_method.map({
                'gradAllReduce': 'All Reduce',
                'noComm': 'No Communication',
                'elasticGossip': 'Elastic Gossip',
                'gossipingSgd': 'Gossiping SGD',
            })) \
    .rename(columns={'rank_0.TestAcc': 'Rank-0 Accuracy',
                     'agg.AggTestAcc': 'Aggregate Accuracy',
                     'agg_method': 'Method', 
                     'agg_prob': 'p',
                     'num_workers': 'W',
                     'elastic_alpha': 'alpha'}) \
    .assign(id=lambda d:
            d.apply(lambda x: '-'.join([
                get_initials(x['Method']),
                str(x['W']) +
                ('-%.3f' % x['p'] if x['p'] == x['p'] else ''),
            ]), axis=1)) \
    .reset_index(drop=True)
    
display(case_1)

print (case_1 \
           [['Method', 'W', 'p', 'Rank-0 Accuracy', 'Aggregate Accuracy']] \
           .to_latex(index=False, na_rep='-'))

display(case_1[['exp_id', 'id']].values.tolist())

,Method,W,p,alpha,Rank-0 Accuracy,exp_id,Aggregate Accuracy,id
0,All Reduce,4,NaN,NaN,0.9193,cifar10_gar_4,0.9193,AR-4
1,Elastic Gossip,4,0.125000,0.5,0.9166,cifar10_elastic_gossip_aggprob8_alpha05_4,0.9146,EG-4-0.125
2,Gossiping SGD,4,0.125000,NaN,0.9131,cifar10_gopr_aggprob8_alpha05_4,0.9135,GS-4-0.125
3,Elastic Gossip,4,0.031250,0.5,0.9122,cifar10_elastic_gossip_aggprob32_alpha05_4,0.9139,EG-4-0.031
4,Gossiping SGD,4,0.031250,NaN,0.9048,cifar10_gopr_aggprob32_alpha05_4,0.9065,GS-4-0.031
5,Elastic Gossip,4,0.007812,0.5,0.9006,cifar10_elastic_gossip_aggprob128_alpha05_4,0.9044,EG-4-0.008
6,Gossiping SGD,4,0.007812,NaN,0.9015,cifar10_gopr_aggprob128_alpha05_4,0.9050,GS-4-0.008
7,Elastic Gossip,4,0.001953,0.5,0.8952,cifar10_elastic_gossip_aggprob512_alpha05_4,0.8983,EG-4-0.002
8,Gossiping SGD,4,0.001953,NaN,0.8825,cifar10_gopr_aggprob512_alpha05_4,0.8845,GS-4-0.002


\begin{tabular}{lrrrr}
\toprule
         Method &  W &         p &  Rank-0 Accuracy &  Aggregate Accuracy \\
\midrule
     All Reduce &  4 &         - &           0.9193 &              0.9193 \\
 Elastic Gossip &  4 &  0.125000 &           0.9166 &              0.9146 \\
  Gossiping SGD &  4 &  0.125000 &           0.9131 &              0.9135 \\
 Elastic Gossip &  4 &  0.031250 &           0.9122 &              0.9139 \\
  Gossiping SGD &  4 &  0.031250 &           0.9048 &              0.9065 \\
 Elastic Gossip &  4 &  0.007812 &           0.9006 &              0.9044 \\
  Gossiping SGD &  4 &  0.007812 &           0.9015 &              0.9050 \\
 Elastic Gossip &  4 &  0.001953 &           0.8952 &              0.8983 \\
  Gossiping SGD &  4 &  0.001953 &           0.8825 &              0.8845 \\
\bottomrule
\end{tabular}



[['cifar10_gar_4', 'AR-4'],
 ['cifar10_elastic_gossip_aggprob8_alpha05_4', 'EG-4-0.125'],
 ['cifar10_gopr_aggprob8_alpha05_4', 'GS-4-0.125'],
 ['cifar10_elastic_gossip_aggprob32_alpha05_4', 'EG-4-0.031'],
 ['cifar10_gopr_aggprob32_alpha05_4', 'GS-4-0.031'],
 ['cifar10_elastic_gossip_aggprob128_alpha05_4', 'EG-4-0.008'],
 ['cifar10_gopr_aggprob128_alpha05_4', 'GS-4-0.008'],
 ['cifar10_elastic_gossip_aggprob512_alpha05_4', 'EG-4-0.002'],
 ['cifar10_gopr_aggprob512_alpha05_4', 'GS-4-0.002']]